In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt

import cv2
import matplotlib.image as mpimg
import PIL
import PIL.Image

In [ ]:
styles_df = pd.read_csv("/kaggle/input/styles-image-path/styles_image_paths.csv", nrows=5000)
image_embeddings = pd.read_csv("/kaggle/input/embeddings-advanced/emb_matrix_new.csv")
text_embeddings = pd.read_csv("/kaggle/input/text-embeddings/textual_embeddings_data.csv")

In [ ]:
text_embeddings.drop('id', inplace=True, axis=1)

In [ ]:
def get_similarity(embedding_df):
    # Compute pairwise cosine similarities
    cosine_similarities = cosine_similarity(embedding_df)

    # Find top n similar products for each product
    n = 5
    similar_products = {}
    for i, row in enumerate(cosine_similarities):
        # Sort indices in descending order based on cosine similarity values and exclude the first index (itself)
        similar_indices = np.argsort(-row)[1:n+1]
        # Get the IDs of the top n similar products
        similar_ids = embedding_df.iloc[similar_indices].index.tolist()
        similar_products[embedding_df.index[i]] = similar_ids
    
    return similar_products


In [ ]:
text_similarity = get_similarity(text_embeddings)
image_similarity = get_similarity(image_embeddings)

In [ ]:
text_similarity[0], image_similarity[0]

In [ ]:
def load_image(img_path, resized_fac = 0.1):
    img_object = plt.imread(img_path)
    w, h, c = img_object.shape
    resized = cv2.resize(img_object, (int(h*resized_fac), int(w*resized_fac)))
    return resized

def plot_image(image_id, styles_df):
    plt.imshow(load_image(styles_df.iloc[image_id]['image']))
    plt.title(styles_df.iloc[image_id]['productDisplayName'])
    
    
def plot_similar_images(query_id, similarity_dict, styles_df=styles_df):
    plot_image(query_id, styles_df=styles_df)
    
    fig = plt.figure(figsize=(40, 10))
    
    pos = 1
    similar_products = similarity_dict[query_id]
    for id in similar_products:
        ax = plt.subplot(1, 5, pos)
        pos += 1
        plt.imshow(load_image(styles_df.iloc[id]['image']))
        plt.title(f"{id}", fontsize=12)
    plt.show()

In [ ]:
print("Text Based Recommendations")
plot_similar_images(0, text_similarity, styles_df=styles_df)
print("Image Based Recommendations")
plot_similar_images(0, image_similarity, styles_df=styles_df)

In [ ]:
print("Text Based Recommendations")
plot_similar_images(10, text_similarity, styles_df=styles_df)
print("Image Based Recommendations")
plot_similar_images(10, image_similarity, styles_df=styles_df)